In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score


from catboost import Pool, CatBoostClassifier

%matplotlib inline
sns.set(style='darkgrid', font_scale=1.5)

### Подготовка данных

In [2]:
data = pd.read_csv('metrics_data.csv', index_col='_id')
person = pd.read_csv('../person.csv', index_col='_id')

In [3]:
data.head()

,x_mean,x_std,x_max,x_min,y_mean,y_std,y_max,y_min,z_mean,z_std,...,sma,x_energy,y_energy,z_energy,x_iqr,y_iqr,z_iqr,xy_corr,xz_corr,yz_corr
_id,,,,,,,,,,,,,,,,,,,,,
1,13110.861519,1489.074033,17584.0,628.0,-231.045310,1249.368959,3960.0,-5076.0,-9781.608168,1204.070214,...,16514.114425,1.741106e+08,1.613309e+06,9.712872e+07,684.0,1788.0,808.0,0.389405,0.895658,0.367969
2,10398.993311,636.795316,11924.0,8728.0,1220.230769,712.538755,3100.0,-1344.0,-12743.739130,516.853783,...,16528.702152,1.085442e+08,1.996250e+06,1.626698e+08,1136.0,735.0,866.0,-0.186981,0.950490,-0.062792
3,12969.669749,807.711275,15236.0,10760.0,-36.354029,1327.785100,4640.0,-5000.0,-10092.491413,1051.603116,...,16538.716131,1.688639e+08,1.762006e+06,1.029628e+08,1164.0,964.0,1368.0,-0.023526,0.921424,-0.023458
6,12765.113725,3289.505788,29072.0,6396.0,815.027451,2927.223542,12488.0,-5844.0,-10411.811765,3428.969363,...,16912.948421,1.737265e+08,9.199305e+06,1.201175e+08,3768.0,3326.0,4846.0,0.078659,-0.394979,-0.021597
7,15146.131868,160.385778,16424.0,12984.0,-1865.611722,122.331084,-1588.0,-3500.0,-6243.567766,290.745677,...,16491.939761,2.294310e+08,3.495458e+06,3.906659e+07,196.0,132.0,485.0,-0.203383,0.750925,-0.452449


In [4]:
person.head()

,height,mass,position,is_valid,age,sex
_id,,,,,,
1,180,85,2,0,18,1
2,185,75,1,0,18,1
3,185,75,2,0,18,1
6,185,75,3,0,18,1
7,185,75,1,1,18,1


In [5]:
data = pd.concat([data, person], axis=1).drop(['is_valid', 'age'], axis=1)

In [6]:
data.head()

,x_mean,x_std,x_max,x_min,y_mean,y_std,y_max,y_min,z_mean,z_std,...,x_iqr,y_iqr,z_iqr,xy_corr,xz_corr,yz_corr,height,mass,position,sex
_id,,,,,,,,,,,,,,,,,,,,,
1,13110.861519,1489.074033,17584.0,628.0,-231.045310,1249.368959,3960.0,-5076.0,-9781.608168,1204.070214,...,684.0,1788.0,808.0,0.389405,0.895658,0.367969,180,85,2,1
2,10398.993311,636.795316,11924.0,8728.0,1220.230769,712.538755,3100.0,-1344.0,-12743.739130,516.853783,...,1136.0,735.0,866.0,-0.186981,0.950490,-0.062792,185,75,1,1
3,12969.669749,807.711275,15236.0,10760.0,-36.354029,1327.785100,4640.0,-5000.0,-10092.491413,1051.603116,...,1164.0,964.0,1368.0,-0.023526,0.921424,-0.023458,185,75,2,1
6,12765.113725,3289.505788,29072.0,6396.0,815.027451,2927.223542,12488.0,-5844.0,-10411.811765,3428.969363,...,3768.0,3326.0,4846.0,0.078659,-0.394979,-0.021597,185,75,3,1
7,15146.131868,160.385778,16424.0,12984.0,-1865.611722,122.331084,-1588.0,-3500.0,-6243.567766,290.745677,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,1,1


In [7]:
y = data['position']
X = data.drop('position', axis=1)

In [8]:
cat_feature = [X.columns.get_loc('sex')]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=1234
)

train_pool = Pool(X_train, label=y_train, cat_features=cat_feature, 
                  feature_names=list(X_train.columns))
test_pool = Pool(X_test, label=y_test, cat_features=cat_feature, 
                 feature_names=list(X_test.columns))

In [10]:
model = CatBoostClassifier(eval_metric='Accuracy', 
                           use_best_model=True, random_seed=0, loss_function='MultiClass')

model.fit(train_pool, eval_set=test_pool, metric_period=100)

Learning rate set to 0.102876
0:	learn: 0.8426966	test: 0.7435897	best: 0.7435897 (0)	total: 53.7ms	remaining: 53.6s
100:	learn: 1.0000000	test: 0.7948718	best: 0.7948718 (100)	total: 164ms	remaining: 1.46s
200:	learn: 1.0000000	test: 0.8205128	best: 0.8205128 (200)	total: 293ms	remaining: 1.17s
300:	learn: 1.0000000	test: 0.8461538	best: 0.8461538 (300)	total: 399ms	remaining: 927ms
400:	learn: 1.0000000	test: 0.8717949	best: 0.8717949 (400)	total: 512ms	remaining: 765ms
500:	learn: 1.0000000	test: 0.8461538	best: 0.8717949 (400)	total: 628ms	remaining: 625ms
600:	learn: 1.0000000	test: 0.8717949	best: 0.8717949 (400)	total: 812ms	remaining: 539ms
700:	learn: 1.0000000	test: 0.8461538	best: 0.8717949 (400)	total: 919ms	remaining: 392ms
800:	learn: 1.0000000	test: 0.8717949	best: 0.8717949 (400)	total: 1.07s	remaining: 267ms
900:	learn: 1.0000000	test: 0.8717949	best: 0.8717949 (400)	total: 1.18s	remaining: 130ms
999:	learn: 1.0000000	test: 0.8461538	best: 0.8717949 (400)	total: 1.31s	

In [11]:
y_pred = model.predict(X_test)

In [12]:
y_pred.ravel()

array([3, 1, 3, 1, 2, 3, 3, 3, 1, 1, 3, 1, 3, 1, 3, 1, 2, 1, 2, 3, 3, 3,
       1, 1, 3, 1, 1, 2, 3, 3, 3, 1, 2, 2, 1, 2, 3, 3, 1])

In [13]:
y_test.to_numpy()

array([3, 1, 3, 1, 2, 3, 3, 2, 1, 1, 3, 1, 3, 2, 3, 1, 2, 1, 2, 3, 3, 3,
       2, 1, 3, 1, 1, 2, 3, 3, 3, 1, 1, 2, 1, 2, 3, 3, 2])

Попробуем предсказать горбатость. $pos=1$

In [24]:
data = pd.read_csv('metrics_data.csv', index_col='_id')
person = pd.read_csv('../person.csv', index_col='_id')

In [25]:
data = pd.concat([data, person], axis=1).drop(['age'], axis=1)

In [26]:
data = data[data['position'] == 1].drop('position', axis=1)

In [27]:
data.head()

,x_mean,x_std,x_max,x_min,y_mean,y_std,y_max,y_min,z_mean,z_std,...,x_iqr,y_iqr,z_iqr,xy_corr,xz_corr,yz_corr,height,mass,is_valid,sex
_id,,,,,,,,,,,,,,,,,,,,,
2,10398.993311,636.795316,11924.0,8728.0,1220.230769,712.538755,3100.0,-1344.0,-12743.739130,516.853783,...,1136.0,735.0,866.0,-0.186981,0.950490,-0.062792,185,75,0,1
7,15146.131868,160.385778,16424.0,12984.0,-1865.611722,122.331084,-1588.0,-3500.0,-6243.567766,290.745677,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,1,1
10,6096.206406,726.284613,9740.0,3788.0,1230.916370,1313.228744,5552.0,-3732.0,-15212.756228,259.461201,...,864.0,1521.0,292.0,-0.633548,0.701143,-0.198193,178,68,0,1
15,14093.322866,141.593488,15084.0,13064.0,-1838.158979,319.296750,-272.0,-4304.0,-8315.038273,212.695382,...,138.0,352.0,196.0,0.312937,0.742702,-0.026072,178,68,1,1
19,11686.384106,388.529873,12736.0,10696.0,833.812677,164.704139,1548.0,408.0,-11611.069063,383.644697,...,732.0,212.0,712.0,0.055515,0.890701,0.068085,182,68,0,1


In [28]:
y = data['is_valid']
X = data.drop('is_valid', axis=1)

In [29]:
cat_feature = [X.columns.get_loc('sex')]

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=1234
)

train_pool = Pool(X_train, label=y_train, cat_features=cat_feature, 
                  feature_names=list(X_train.columns))
test_pool = Pool(X_test, label=y_test, cat_features=cat_feature, 
                 feature_names=list(X_test.columns))

In [31]:
model = CatBoostClassifier(eval_metric='Accuracy', 
                           use_best_model=True, random_seed=0)

model.fit(train_pool, eval_set=test_pool, metric_period=100)

Learning rate set to 0.013217
0:	learn: 1.0000000	test: 0.7692308	best: 0.7692308 (0)	total: 4.48ms	remaining: 4.47s
100:	learn: 1.0000000	test: 0.9230769	best: 0.9230769 (100)	total: 60ms	remaining: 534ms
200:	learn: 1.0000000	test: 0.9230769	best: 0.9230769 (100)	total: 97.5ms	remaining: 387ms
300:	learn: 1.0000000	test: 0.8461538	best: 0.9230769 (100)	total: 136ms	remaining: 317ms
400:	learn: 1.0000000	test: 0.9230769	best: 0.9230769 (100)	total: 173ms	remaining: 259ms
500:	learn: 1.0000000	test: 0.9230769	best: 0.9230769 (100)	total: 226ms	remaining: 226ms
600:	learn: 1.0000000	test: 0.9230769	best: 0.9230769 (100)	total: 262ms	remaining: 174ms
700:	learn: 1.0000000	test: 0.9230769	best: 0.9230769 (100)	total: 296ms	remaining: 126ms
800:	learn: 1.0000000	test: 0.9230769	best: 0.9230769 (100)	total: 334ms	remaining: 83.1ms
900:	learn: 1.0000000	test: 0.9230769	best: 0.9230769 (100)	total: 372ms	remaining: 40.9ms
999:	learn: 1.0000000	test: 0.9230769	best: 0.9230769 (100)	total: 410m

In [32]:
for value, name in sorted(
    zip(model.get_feature_importance(fstr_type="FeatureImportance"), 
        X_train.columns), 
    reverse=True
):
    print("{}\t{:.2f}".format(name, value))

y_energy	10.82
z_min	8.73
x_energy	7.82
z_mean	6.39
xz_corr	6.38
x_mean	5.64
sma	4.98
x_min	4.91
z_max	4.09
yz_corr	4.05
z_energy	3.91
x_iqr	3.90
x_max	3.65
y_iqr	3.21
y_max	3.07
mass	2.63
xy_corr	2.29
y_min	2.03
z_iqr	2.03
y_std	1.89
sex	1.83
z_std	1.68
y_mean	1.62
x_std	1.26
height	1.21
